<a href="https://colab.research.google.com/github/kiplangatkorir/MetaLearning/blob/main/meta_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/conedata.csv')
data.head()

Mounted at /content/drive


,ID,test_type,basin_valley,Depth (m),qc (MPa),fs (kPa),u2 (kPa),Vs (m/s),qt (MPa),Rf (%),...,Fr (%),Bq (-),U2 (-),SBT (-),SBTn (-),Mod. SBTn (-),n,Ic (-),Ic SBT (-),Ib (-)
0,0,CPT,Zell basin,0.02,1.15,1.0,0,0,1.77,0.06,...,0.06,0,0,0,0,0,-0.12,2.56,2.23,25.61
1,0,CPT,Zell basin,0.04,2.85,1.0,0,0,2.99,0.03,...,0.03,0,0,0,0,0,0.33,1.23,2.02,256.60
2,0,CPT,Zell basin,0.06,4.80,1.0,0,0,4.76,0.06,...,0.06,0,0,0,0,0,0.28,1.16,1.80,264.66
3,0,CPT,Zell basin,0.08,6.40,6.0,0,0,6.12,0.07,...,0.07,0,0,0,0,0,0.27,1.12,1.69,275.17
4,0,CPT,Zell basin,0.10,6.90,6.0,0,0,6.91,0.10,...,0.10,0,0,0,0,0,0.27,1.11,1.65,267.92


In [2]:
!pip install learn2learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 36.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 75.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.8 

In [3]:
!pip install --upgrade learn2learn

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split




In [12]:
# Features
data = pd.read_csv('/content/drive/MyDrive/conedata.csv')
X = data[['qc (MPa)', 'fs (kPa)', 'u2 (kPa)', 'Vs (m/s)']]

# Target variable
y = data['SBT (-)']

In [13]:
data.head()
data.info()
data.columns
(data.shape)
(data.describe())
(data.isnull().sum())
(data.dtypes)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2876 entries, 0 to 2875
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             2876 non-null   int64  
 1   test_type      2876 non-null   object 
 2   basin_valley   2876 non-null   object 
 3   Depth (m)      2876 non-null   float64
 4   qc (MPa)       2876 non-null   float64
 5   fs (kPa)       2876 non-null   float64
 6   u2 (kPa)       2876 non-null   int64  
 7   Vs (m/s)       2876 non-null   int64  
 8   qt (MPa)       2876 non-null   float64
 9   Rf (%)         2876 non-null   float64
 10  γ (kN/m³)      2876 non-null   int64  
 11  σ,v (kPa)      2876 non-null   float64
 12  u0 (kPa)       2876 non-null   float64
 13  σ',v (kPa)     2876 non-null   float64
 14  Qt (-)         2876 non-null   float64
 15  Qtn (-)        2876 non-null   float64
 16  Fr (%)         2876 non-null   float64
 17  Bq (-)         2876 non-null   int64  
 18  U2 (-)  

ID                 int64
test_type         object
basin_valley      object
Depth (m)        float64
qc (MPa)         float64
fs (kPa)         float64
u2 (kPa)           int64
Vs (m/s)           int64
qt (MPa)         float64
Rf (%)           float64
γ (kN/m³)          int64
σ,v (kPa)        float64
u0 (kPa)         float64
σ',v (kPa)       float64
Qt (-)           float64
Qtn (-)          float64
Fr (%)           float64
Bq (-)             int64
U2 (-)             int64
SBT (-)            int64
SBTn (-)           int64
Mod. SBTn (-)      int64
n                float64
Ic (-)           float64
Ic SBT (-)       float64
Ib (-)           float64
dtype: object

In [25]:
# Standardize features (optional but recommended)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32).unsqueeze(1)

In [26]:
# Define your neural network architecture
class CPTMetaLearner(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(CPTMetaLearner, self).__init__()
        # Define layers (e.g., fully connected layers)
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, output_dim)

    def forward(self, x):
        # Forward pass
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [27]:
# Initialize meta-learner
input_dim = X_tensor.shape[1]
output_dim = 1
meta_model = CPTMetaLearner(input_dim, output_dim)
optimizer = optim.Adam(meta_model.parameters(), lr=0.001)


In [28]:
# Split data into meta-training and meta-testing tasks
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

In [29]:
# Meta-training loop
num_meta_epochs = 10
for _ in range(num_meta_epochs):
    # Fine-tune meta-learner on task_train_data
    optimizer.zero_grad()
    y_pred = meta_model(X_train)
    loss = nn.MSELoss()(y_pred, y_train)
    loss.backward()
    optimizer.step()


In [30]:
# Meta-testing loop
y_pred_test = meta_model(X_test)
test_loss = nn.MSELoss()(y_pred_test, y_test)

print(f"Meta-testing loss: {test_loss.item()}")

Meta-testing loss: 20.998178482055664
